In [1]:
# %edit
# %load_ext autoreload
# %autoreload 2

IPython will make a temporary file named: /tmp/ipython_edit_auu67nfg/ipython_edit_j_tals56.py


In [16]:
# GPT-2 code was run in April, 2023
import openai
# import jax.numpy as jnp
# from jax.scipy.special import logsumexp
from typing import Optional, Tuple, Dict
import numpy as np
import pandas as pd
import time
# import hydra
# from cdt.causality.pairwise import RECI
import os
# from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer
import re
from diskcache import Cache
from tqdm import tqdm
import json
import time

import torch

from transformers import pipeline, set_seed
from transformers import GPT2Tokenizer, GPT2Model

from multiprocessing import Pool


from utils import *

In [17]:
engine = "gpt2-xl"  # "gpt2-large"
# load model
model = GPT2LMHeadModel.from_pretrained(engine)
tokenizer = GPT2Tokenizer.from_pretrained(engine)

In [23]:
recog_prompt = """This is a set of location description recognition problems.
The `Sentence` is a sentence containing location descriptions.
The goal is to infer which parts of the sentence represent location descriptions, their categories, and split different location descriptions with `;`.
--

--
Sentence: Streets Flooded: Almeda Genoa Rd. from Windmill Lakes Blvd. to Rowlett Rd. HurricaneHarvey Houston
Q: Which parts of this sentence represent location descriptions?
A: C11: Almeda Genoa Rd. from Windmill Lakes Blvd. to Rowlett Rd.; C9: Houston
--

--
Sentence: Anyone doing high water rescues in the Pasadena/Deer Park area? My daughter has been stranded in a parking lot all night
Q: Which parts of this sentence represent location descriptions?
A: C10: Pasadena/Deer Park
--

--
Sentence: Allen Parkway, Memorial, Waugh overpass, Spotts park and Buffalo Bayou park completely under water
Q: Which parts of this sentence represent location descriptions?
A: C2: Allen Parkway; C2: Memorial; C2: Waugh overpass; C7: Spotts park; C7: Buffalo Bayou park
--

--
Sentence: 9:00AM update video from Hogan St over White Oak Bayou, I-10, I-45: water down about 4' since last night
Q: Which parts of this sentence represent location descriptions?
A: C5: Hogan St over White Oak Bayou; C3: I-10; C3: I-45
--

--
Sentence: Need trailers/trucks to move dogs from Park Location: Whites Park Pavillion off I-10 exit 61 Anahuac TX
Q: Which parts of this sentence represent location descriptions?
A: C7: Whites Park Pavillion; C3: I-10; C4: exit 61; C9: Anahuac; C9: TX
--

--
Sentence: #Harvey does anyone know about the flooding conditions around Cypress Ridge High School?! #HurricaneHarvey
Q: Which parts of this sentence represent location descriptions?
A: C8: Cypress Ridge High School
--

--
Sentence: If you need a place to escape #HurricaneHarvey, The Willie De Leon Civic Center: 300 E. Main St in Uvalde is open as a shelter
Q: Which parts of this sentence represent location descriptions?
A: C7: The Willie De Leon Civic Center; C1: 300 E. Main St in Uvalde
--

--
Sentence: #HurricaneHarvey INTENSE eye wall of category 4 Hurricane from Rockport, TX
Q: Which parts of this sentence represent location descriptions?
A: C9: Rockport; C9: TX
--

--
Sentence: Major flooding at Clay Rd & Queenston in west Houston. Lots of rescues going on for ppl trapped.
Q: Which parts of this sentence represent location descriptions?
A: C5: Clay Rd & Queenston; C9: Houston
--

--
Sentence: #HurricaneHarvey family needs rescuing at 11800 Grant Rd. Apt. 1009 in Cypress, Texas 77429
Q: Which parts of this sentence represent location descriptions?
A: C1: 11800 Grant Rd. Apt. 1009 in Cypress, Texas 77429
--

--
Sentence: Buffalo Bayou holding steady at 10,000 cfs at the gage near Terry Hershey Park
Q: Which parts of this sentence represent location descriptions?
A: C6: Buffalo Bayou; C7: Terry Hershey Park
--

--
Sentence: {TEXT}
Q: Which parts of this sentence represent location descriptions?
A:"""

In [24]:
len(tokenizer.encode(recog_prompt))

826

In [25]:

# for file in os.listdir(data_dir):
#     if file.endswith(".txt") or file != "tweetTextAndAnnotation.txt":
#         file_path = os.path.join(data_dir, file)
#         break

In [26]:
data_dir = "data/"

anno_file = os.path.join(data_dir, 'tweetTextAndAnnotation.txt')

data_list = load_HarveyTweet2017_data(anno_file, data_dir, sep = "||")

In [27]:
data_list[0]

{'ID': 0,
 'tweetFileID': 'data/0.txt',
 'tweetFulltext': 'Houston HoustonFlood the intersection of I-45 & N. Main Street',
 'tweetAnnotation': [{'locationDesc': 'Houston',
   'locationCate': 'C9',
   'startIndex': 0,
   'endIndex': 7},
  {'locationDesc': 'I-45 & N. Main Street',
   'locationCate': 'C5',
   'startIndex': 41,
   'endIndex': 62}]}

In [7]:
# p_len_list = []
# a_len_list = []
# for item in data_list:
#     text = item["tweetFulltext"]
#     recog_prompt_ = recog_prompt.replace("{TEXT}", text)
#     answer = ( "; ".join([x['locationDesc'] for x in item['tweetAnnotation']])  )
    
#     p_len = len(tokenizer.encode(recog_prompt_))
#     p_len_list.append(p_len)
    
#     a_len = len(tokenizer.encode(answer))
#     a_len_list.append(a_len)
    
# max(p_len_list), max(a_len_list)

In [8]:

n_tokens = 30
temp=0.2
logprobs = 5
logprobs_topk = 20

res_dir = "result/"
if not os.path.exists(res_dir):
    os.mkdir(res_dir)
res_file = "{res_dir}/placerecognize_{engine}_{temp:.3f}_{n_tokens:d}_{logprobs:d}.json".format(
        res_dir = res_dir,
        engine = engine,
        temp = temp,
        n_tokens = n_tokens,
        logprobs = logprobs
    )
res_dict_list = read_res_file(res_file)
process_ids = [res_dict['ID'] for res_dict in res_dict_list]

cur_data_list = [item for item in data_list if item["ID"] not in process_ids]

In [9]:


print(engine, res_file)

gpt2-xl result//placerecognize_gpt2-xl_0.200_30_5.json


In [18]:
def get_place_recognition_answers_with_logprob_HarveyTweet2017(item, recog_prompt, tokenizer, 
                                model,
                                n_tokens = 16, 
                                num_return_sequences = 5,
                                logprobs_topk = 20,
                                temparature = 0):

    text = item["tweetFulltext"]
    ID = item["ID"]

    # text_ = text.capitalize()

    recog_prompt_ = recog_prompt.replace("{TEXT}", text)
    #print(recog_prompt_)

    # n_answer_tokens = len(tokenizer.encode(phrase)) 

    total_length = len(tokenizer.encode(recog_prompt_)) + n_tokens

    inputs = tokenizer(recog_prompt_, return_tensors="pt")

    model.eval()
   
    outputs = model.generate(inputs = inputs['input_ids'], 
                             output_scores = True, 
                             return_dict_in_generate = True,
    #                         do_sample = True,
                             top_k = 0,
    #                          top_p = 0.95,
                             max_length = total_length,
                             eos_token_id = tokenizer.encode("\n")[0],
                            num_return_sequences = 1,
                            temperature = temparature)

    # scores: shape (answer_len, num_vocab)
    scores = outputs['scores']
    scores = torch.cat(scores, dim = 0)
    # normalize the token scores
    logsoftmax = torch.nn.LogSoftmax(dim=-1)
    scores = logsoftmax(scores)

    prompt_len = inputs['input_ids'].shape[-1]
    # get answer token indices
    # answer_ids: int, shape (answer_len)
    answer_ids = outputs["sequences"][0][prompt_len:]

    # answer_logprob: shape (answer_len), answer token probablity
    answer_logprob = scores[np.arange(len(answer_ids)), answer_ids].cpu().numpy().astype(np.float64)
    # answer_tokens: shape (answer_len), answer tokens
    answer_tokens = tokenizer.batch_decode(answer_ids)


    scores_values, scores_ids = torch.sort(scores, descending = True, dim = -1)

    answer_logprobs_dict_list = []
    for i, idx in enumerate(np.arange(len(scores_ids))):
        cur_score_ids_tokens = tokenizer.batch_decode(scores_ids[idx, :logprobs_topk])
        answer_logprobs_dict = dict(zip(cur_score_ids_tokens, scores_values[idx, :logprobs_topk].cpu().numpy().astype(np.float64)))
        answer_logprobs_dict_list.append(answer_logprobs_dict)

    answer_ids = answer_ids.cpu().numpy()

    # answers results
    answers = []
    for answer_seq in tokenizer.batch_decode(outputs['sequences']):
        answers.append(answer_seq.replace(recog_prompt_, "") )


    res_dict = {'answer': ''.join(answer_tokens),
                'answer_tokens': answer_tokens,
                #'answer_logprobs_list': list(answer_logprob),
                #'answer_logprobs': answer_logprobs_dict_list,
                **item
            }
    return res_dict

In [31]:
res_dict = get_place_recognition_answers_with_logprob_HarveyTweet2017(cur_data_list[3], recog_prompt, tokenizer, 
                                model,
                                n_tokens = n_tokens, 
                                num_return_sequences = logprobs,
                                logprobs_topk = logprobs_topk,
                                temparature = temp)

res_dict['answer']

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.


' C5: Rockport; C7: Terry Hershey Park\n'

In [12]:
for i, item in (pbar := tqdm(enumerate(cur_data_list), total= len(cur_data_list) )):
    res_dict = get_place_recognition_answers_with_logprob_HarveyTweet2017(item, recog_prompt, tokenizer, 
                                model,
                                n_tokens = n_tokens, 
                                num_return_sequences = logprobs,
                                logprobs_topk = logprobs_topk,
                                temparature = temp)
    with open(res_file, 'a') as f:
        f.write(json.dumps(res_dict) + "\n")

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.
  0%|                                                                               | 1/1000 [00:26<7:21:38, 26.53s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.
  0%|▏                                                                              | 2/1000 [00:49<6:47:26, 24.50s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_

  4%|███▍                                                                          | 44/1000 [19:36<6:54:54, 26.04s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.
  4%|███▌                                                                          | 45/1000 [20:07<7:19:17, 27.60s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.
  5%|███▌                                                                          | 46/1000 [20:36<7:21:05, 27.74s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_

  9%|██████▊                                                                       | 88/1000 [39:37<6:40:05, 26.32s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.
  9%|██████▉                                                                       | 89/1000 [40:01<6:29:01, 25.62s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.
  9%|███████                                                                       | 90/1000 [40:30<6:44:35, 26.68s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_

 13%|█████████▉                                                                 | 132/1000 [1:13:38<6:41:44, 27.77s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.
 13%|█████████▉                                                                 | 133/1000 [1:14:06<6:38:46, 27.60s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.
 13%|██████████                                                                 | 134/1000 [1:14:38<7:00:28, 29.13s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_

 18%|█████████████▏                                                             | 176/1000 [1:33:54<5:56:52, 25.99s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.
 18%|█████████████▎                                                             | 177/1000 [1:34:27<6:25:29, 28.10s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.
 18%|█████████████▎                                                             | 178/1000 [1:34:54<6:18:45, 27.65s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_

...

 79%|███████████████████████████████████████████████████████████▍               | 792/1000 [7:01:46<1:32:40, 26.73s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.
 79%|███████████████████████████████████████████████████████████▍               | 793/1000 [7:02:12<1:31:05, 26.40s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.
 79%|███████████████████████████████████████████████████████████▌               | 794/1000 [7:02:40<1:32:35, 26.97s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_

 84%|██████████████████████████████████████████████████████████████▋            | 836/1000 [7:20:59<1:18:26, 28.70s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.
 84%|██████████████████████████████████████████████████████████████▊            | 837/1000 [7:21:23<1:13:41, 27.13s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.
 84%|██████████████████████████████████████████████████████████████▊            | 838/1000 [7:21:48<1:12:09, 26.72s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_

 88%|███████████████████████████████████████████████████████████████████▊         | 880/1000 [7:40:33<52:33, 26.28s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.
 88%|███████████████████████████████████████████████████████████████████▊         | 881/1000 [7:40:59<52:01, 26.23s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.
 88%|███████████████████████████████████████████████████████████████████▉         | 882/1000 [7:41:27<52:23, 26.64s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_

 92%|███████████████████████████████████████████████████████████████████████▏     | 924/1000 [8:00:06<32:15, 25.47s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.
 92%|███████████████████████████████████████████████████████████████████████▏     | 925/1000 [8:00:33<32:25, 25.94s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.
 93%|███████████████████████████████████████████████████████████████████████▎     | 926/1000 [8:00:58<31:38, 25.65s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_

 97%|██████████████████████████████████████████████████████████████████████████▌  | 968/1000 [8:19:56<14:40, 27.53s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.
 97%|██████████████████████████████████████████████████████████████████████████▌  | 969/1000 [8:20:25<14:25, 27.93s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.
 97%|██████████████████████████████████████████████████████████████████████████▋  | 970/1000 [8:20:57<14:36, 29.23s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_

In [63]:
cur_data_list[2]

{'ID': 249,
 'tweetFileID': 'data/322.txt',
 'tweetFulltext': 'Beaumont, TX Is Now A Lake From Harvey Hurricane',
 'tweetAnnotation': [{'locationDesc': 'Beaumont',
   'locationCate': 'C9',
   'startIndex': 0,
   'endIndex': 8},
  {'locationDesc': 'TX',
   'locationCate': 'C9',
   'startIndex': 10,
   'endIndex': 12}]}

### GPT-2 without Geo-knowledge

In [19]:
recog_prompt = """
This is a set of location description recognition problems. 
The `Sentence` is a sentence containing location descriptions. The goal is to infer which parts of the sentence represent location descriptions and put them in double quotes.
--

--
Sentence: {TEXT}
Q: Which parts of this sentence represent location descriptions?
A:"""

In [20]:
data_dir = "data/"

anno_file = os.path.join(data_dir, 'tweetTextAndAnnotation.txt')

data_list = load_HarveyTweet2017_data(anno_file, data_dir, sep = "||")

In [21]:
n_tokens = 30
temp=0.2
logprobs = 5
logprobs_topk = 20

res_dir = "result/"
if not os.path.exists(res_dir):
    os.mkdir(res_dir)
res_file = "{res_dir}/placerecognize_{engine}_{temp:.3f}_{n_tokens:d}_{logprobs:d}_no_geo.json".format(
        res_dir = res_dir,
        engine = engine,
        temp = temp,
        n_tokens = n_tokens,
        logprobs = logprobs
    )
res_dict_list = read_res_file(res_file)
process_ids = [res_dict['ID'] for res_dict in res_dict_list]

cur_data_list = [item for item in data_list if item["ID"] not in process_ids]
len(cur_data_list)

1000

In [22]:
count = 0

for i, item in (pbar := tqdm(enumerate(cur_data_list), total= len(cur_data_list) )):
    res_dict = get_place_recognition_answers_with_logprob_HarveyTweet2017(item, recog_prompt, tokenizer, 
                                model,
                                n_tokens = n_tokens, 
                                num_return_sequences = logprobs,
                                logprobs_topk = logprobs_topk,
                                temparature = temp)
    
    
    original_answer = res_dict['answer'].strip(" \n")
    if "\"" in original_answer:
        extracted_answer = re.findall('"([^"]*)"', original_answer)
    else:
        extracted_answer = original_answer
    
    res_dict['origin_answer'] = original_answer
    if type(extracted_answer) is list:
        res_dict['answer'] = ";".join(["CX: "+str(this_str) for this_str in extracted_answer])
    elif len(extracted_answer)>0:
        res_dict['answer'] = "CX: "+extracted_answer
    
    with open(res_file, 'a') as f:
        f.write(json.dumps(res_dict) + "\n")
        
    # print the result for a preview
    print("Tweet ID is: "+ res_dict['tweetFileID'])
    print("Tweet text is: "+ res_dict['tweetFulltext'])
    print("Annotation is: "+ str(res_dict['tweetAnnotation']))
    print("Original answer is: "+ res_dict['origin_answer'])
    print("Answer is: "+ res_dict['answer'])
    
    count += 1
    #time.sleep(0.5)
    
#     if count == 5:
#         break

f.close()

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.
  0%|                                                                               | 1/1000 [00:07<1:59:15,  7.16s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.


Tweet ID is: data/0.txt
Tweet text is: Houston HoustonFlood the intersection of I-45 & N. Main Street
Annotation is: [{'locationDesc': 'Houston', 'locationCate': 'C9', 'startIndex': 0, 'endIndex': 7}, {'locationDesc': 'I-45 & N. Main Street', 'locationCate': 'C5', 'startIndex': 41, 'endIndex': 62}]
Original answer is: I-45 & N. Main Street
Answer is: CX: I-45 & N. Main Street


  0%|▏                                                                              | 2/1000 [00:22<3:18:40, 11.94s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.


Tweet ID is: data/1.txt
Tweet text is: HarveyStorm over Austin, TX at 8: 00 AM CDT via Weather Underground
Annotation is: [{'locationDesc': 'Austin', 'locationCate': 'C9', 'startIndex': 17, 'endIndex': 23}, {'locationDesc': 'TX', 'locationCate': 'C9', 'startIndex': 25, 'endIndex': 27}]
Original answer is: The first part of the sentence is a location description. The second part is a sentence containing location descriptions.
Answer is: CX: The first part of the sentence is a location description. The second part is a sentence containing location descriptions.


  0%|▏                                                                              | 3/1000 [00:24<2:07:08,  7.65s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.


...

Tweet ID is: data/997.txt
Tweet text is: Teachers from the Houston area are collecting supplies for Harvey at Sara Park in Tomahawk until 7p. m.
Annotation is: [{'locationDesc': 'Houston', 'locationCate': 'C9', 'startIndex': 18, 'endIndex': 25}, {'locationDesc': 'Sara Park', 'locationCate': 'C7', 'startIndex': 69, 'endIndex': 78}, {'locationDesc': 'Tomahawk', 'locationCate': 'C9', 'startIndex': 82, 'endIndex': 90}]
Original answer is: The first part of the sentence is a location description. The second part is a sentence containing location descriptions.
Answer is: CX: The first part of the sentence is a location description. The second part is a sentence containing location descriptions.


100%|████████████████████████████████████████████████████████████████████████████▉| 999/1000 [1:19:04<00:03,  3.69s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.


Tweet ID is: data/998.txt
Tweet text is: Teen boy was rescued and his mother was present. Harvey2017 South Shore Park
Annotation is: [{'locationDesc': 'South Shore Park', 'locationCate': 'C7', 'startIndex': 60, 'endIndex': 76}]
Original answer is: 
Answer is: 



100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:19:10<00:00,  4.75s/it]

Tweet ID is: data/999.txt
Tweet text is: Texans helping Texans is what is going to help bring relief to harvey victims. Head down to 1800 N. Lamar St. and â€¦
Annotation is: [{'locationDesc': '1800 N. Lamar St.', 'locationCate': 'C1', 'startIndex': 92, 'endIndex': 109}]
Original answer is: The part that starts with "Texans" is the part that represents the location.
Answer is: CX: Texans
